In [1]:
import numpy as np
import pandas as pd

In [2]:
dir ='/root/JupyterNotebook/kaggle-Redefining-Cancer-Treatment'

In [3]:
train_txt = pd.read_csv(dir+'/4 input for stage2/training_txt.csv')

In [4]:
test_txt = pd.read_csv(dir+'/4 input for stage2/test_txt.csv')

In [5]:
# define filter word function
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sw

stopWords = ['fig','figure', 'et', 'al', 'table', 'i.e', 'i.e.'] + list(sw)

def isnumeric(s):
    '''returns True if string s is numeric'''
    return all(c in "0123456789.+-,≥≤–/±" for c in s)

def filterword(wordlist):
    result = []
    for word in wordlist:
        if (word not in stopWords) and (not isnumeric(word)) and len(word)>2:
            result.append(word)
    return result

In [6]:
# re for special split sentence
import re
redotrepl1 = re.compile(r'[a-zA-Z].\d\s')
redotrepl3 = re.compile(r'.\(\d')
def dotrepl1(matchobj):
    return matchobj.group(0).replace('.', '. ')
redotrepl2 = re.compile(r'.\d.\d\s')
def dotrepl2(matchobj):
    s = matchobj.group(0)[::-1]
    s = s.replace('.', ' .',1)
    return s[::-1]
refigrepl1 =re.compile(r'[fF]ig[ure]{0,3}\s\d.')
refigrepl2 =re.compile(r'[fF]ig[ure]{0,3}.')
def figrepl(matchobj):
    return ' '
def reall(str_in):
    stemp = redotrepl1.sub(dotrepl1, str_in)
    stemp = redotrepl2.sub(dotrepl2, stemp)
    stemp = redotrepl3.sub(dotrepl1, stemp)
    stemp = refigrepl1.sub(figrepl, stemp)
    stemp = refigrepl2.sub(figrepl, stemp)
    return stemp

In [7]:
# define clean function
from textblob import TextBlob
from textblob import Word

def cleandoc(doc):
    doc = reall(doc)
    doc = TextBlob(doc)
    sentences = doc.sentences
    txt = []
    for st in sentences:
        st = st.lower()       
        tag = st.tags
        w_list = []
        for word,pos in tag:
            if pos[0:2] == 'VB':
                w_list.append(word.lemmatize("v"))
            else:
                w_list.append(word.lemmatize())
        fw = filterword(w_list)
        sent = ' '.join(fw)
        txt.append(sent)
    result = ' . '.join(txt)
    if result == '':
        result = 'none'     
    return result

In [8]:
#train
temp_doc = pd.Series(train_txt.apply(lambda r:cleandoc(r['Text']), axis=1),name='CleanText')
train_cleantxt = pd.concat([train_txt['ID'],temp_doc],axis=1)
train_cleantxt.to_csv(dir+'/5 middle for stage2/train_cleantxt.txt', index=False)

#test
temp_doc = pd.Series(test_txt.apply(lambda r:cleandoc(r['Text']), axis=1),name='CleanText')
test_cleantxt = pd.concat([test_txt['ID'],temp_doc],axis=1)
test_cleantxt.to_csv(dir+'/5 middle for stage2/test_cleantxt.txt', index=False)